#### Reducción proporcional de valores usando Python
##### Debido al margen de error del modelo de procesos para obtener datos censales por barrios, es necesario ajustar los valores censales totales a los oficiales, reduciendo proporcionalmente cada valor.

In [ ]:
#Cargar librerías
import pandas as pd

In [ ]:
# Cargar y leer archivo
df_pob = pd.read_csv("join_barrios_x_link_pob_csv_listo.csv")
df_pob.head(2)

,nombre,pob
0,Chacras Oeste,1564
1,"Chacras Tres Pinos, Cañadones",362


In [ ]:
# Definir valores de referencia
total_original = 203290
total_objetivo = 201228
diferencia = total_original - total_objetivo

In [ ]:
# Calcular la proporción de reducción de cada barrio

df_pob["proporcion"] = df_pob["pob"] / total_original
df_pob["reduccion"] = df_pob["proporcion"] * diferencia

In [18]:
df_pob.head(2)

,nombre,pob,proporcion,reduccion
0,Chacras Oeste,1564,0.007693,15.863879
1,"Chacras Tres Pinos, Cañadones",362,0.001781,3.671819


In [ ]:
# Redondear hacia el número más cercano

df_pob["reduccion_redondeada"] =df_pob["reduccion"].round()

In [ ]:
# Ver los resultados después de redondear

df_pob[["nombre", "pob", "reduccion", "reduccion_redondeada"]].head(2)

,nombre,pob,reduccion,reduccion_redondeada
0,Chacras Oeste,1564,15.863879,16.0
1,"Chacras Tres Pinos, Cañadones",362,3.671819,4.0


In [ ]:
# Sumar las reducciones redondeadas

total_reduccion_redondeada = df_pob["reduccion_redondeada"].sum()

In [ ]:
# Calcular la diferencia

ajuste = diferencia - total_reduccion_redondeada

In [ ]:
# Realizar el ajuste (agregar o quitar una persona en algunos barrios,
# especialmente aquellos con mayor población)

if ajuste != 0:
    # Ordenar los barrios por población (o por reducción y aplicar el ajuste)
    if ajuste > 0:
        # Agregar personas donde más impacto tiene (por ejemplo, barrios más poblados)
        barrios_ajustar = df_pob.nlargest(ajuste, "pob")
        for index in barrios_ajustar.index:
            df_pob.at[index, "reduccion_redondeada"] += 1
    elif ajuste < 0:
        # Quitar personas donde más impacto tiene (por ejemplo, barrios más poblados)
        barrios_ajustar = df_pob.nlargest(abs(ajuste), "pob")
        for index in barrios_ajustar.index:
            df_pob.at[index, "reduccion_redondeada"] -= 1